def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [20]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [3]:
# bajamos los tres datasets para ver cual es mas conveniente para nuestro metodo de recomendacion
review = pd.read_parquet("clean_review")
items = pd.read_parquet("clean_items")
steam = pd.read_parquet("clean_steam_games")

In [4]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             59333 non-null  object 
 1   item_id             59305 non-null  float64
 2   recommend           59305 non-null  float64
 3   sentiment_analysis  59333 non-null  int64  
 4   Año_posteo          59333 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 2.3+ MB


In [5]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   item_id           float64
 3   item_name         object 
 4   playtime_forever  float64
 5   playtime_2weeks   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 236.7+ MB


In [6]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   item_id                    32133 non-null  int64  
 1   app_name                   32132 non-null  object 
 2   price                      32123 non-null  float64
 3   developer                  28835 non-null  object 
 4   Accounting                 32133 non-null  int64  
 5   Action                     32133 non-null  int64  
 6   Adventure                  32133 non-null  int64  
 7   Animation &amp; Modeling   32133 non-null  int64  
 8   Audio Production           32133 non-null  int64  
 9   Casual                     32133 non-null  int64  
 10  Design &amp; Illustration  32133 non-null  int64  
 11  Early Access               32133 non-null  int64  
 12  Education                  32133 non-null  int64  
 13  Free to Play               32133 non-null  int

### Observamos que solo necesitas la tabla review y la columna de app_name de la tabla steam

In [7]:
review_y_steam = pd.merge(review, steam, on="item_id")

In [8]:
review_y_steam.head(1)

,user_id,item_id,recommend,sentiment_analysis,Año_posteo,app_name,price,developer,Accounting,Action,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,Año_lanzamiento
0,76561197970982479,1250.0,1.0,2,2011,Killing Floor,19.99,Tripwire Interactive,0,1,...,0,0,0,0,0,0,0,0,0,2009.0


### Me quedo con las columnas que voy a utilizar

In [9]:
merge = review_y_steam[["user_id","recommend","sentiment_analysis","app_name"]]

In [10]:
merge["recommend"].value_counts()

recommend
1.0    48206
0.0     5782
Name: count, dtype: int64

In [11]:
merge[merge["user_id"]== "GamerFag"]

,user_id,recommend,sentiment_analysis,app_name
5,GamerFag,1.0,2,Killing Floor
6340,GamerFag,1.0,1,Half-Life 2
6597,GamerFag,1.0,2,Team Fortress 2
30045,GamerFag,1.0,0,Half-Life
33500,GamerFag,1.0,1,Half-Life 2: Episode Two
33519,GamerFag,1.0,2,Half-Life 2: Episode One


In [12]:
for indice, linea in merge.iterrows():
    if (linea[1] == 1) and (linea[2] == 2):
        merge.at[indice, "rating"] = 5
    elif (linea[1] == 1) and (linea[2] == 1):
        merge.at[indice, "rating"] = 4
    elif (linea[1] == 1) and (linea[2] == 0):
        merge.at[indice, "rating"] = 3
    elif (linea[1] == 0) and (linea[2] == 2):
        merge.at[indice, "rating"] = 2
    elif (linea[1] == 0) and (linea[2] == 1):
        merge.at[indice, "rating"] = 1
    elif (linea[1] == 0) and (linea[2] == 0):
        merge.at[indice, "rating"] = 0

C:\Users\Usuario\AppData\Local\Temp\ipykernel_26016\2381378555.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (linea[1] == 1) and (linea[2] == 2):
C:\Users\Usuario\AppData\Local\Temp\ipykernel_26016\2381378555.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge.at[indice, "rating"] = 5
C:\Users\Usuario\AppData\Local\Temp\ipykernel_26016\2381378555.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value

In [14]:
merge.head(1)

,user_id,recommend,sentiment_analysis,app_name,rating
0,76561197970982479,1.0,2,Killing Floor,5.0


In [15]:
modelo = merge[["user_id","app_name","rating"]]

In [16]:
modelo

,user_id,app_name,rating
0,76561197970982479,Killing Floor,5.0
1,death-hunter,Killing Floor,5.0
2,DJKamBer,Killing Floor,3.0
3,diego9031,Killing Floor,4.0
4,76561198081962345,Killing Floor,4.0
...,...,...,...
53983,llDracuwulf,Asteria,5.0
53984,ChrisCoroner,Street Fighter X Tekken,5.0
53985,MeloncraftLP,The Journey Down: Chapter One,4.0
53986,MeloncraftLP,The Journey Down: Chapter Two,4.0


In [36]:
Scala = Reader(rating_scale=(0, 5))  # Adapto el reating a una scala

In [42]:
# Cargo los datos con el lector 
data_nuevo = Dataset.load_from_df(modelo, Scala)

In [43]:
data_nuevo

### Divido la data para entrenarlo

In [112]:
trainset, testset = train_test_split(data_nuevo, test_size=0.25,random_state=42)

### Utilizo "SVD" para poder entrenar el modelo

In [53]:
algoritmo = SVD()
modelo_entrenado = algoritmo.fit(trainset)

In [54]:
modelo_entrenado 

### Guardo el modelo en un archivo pickle para luego utilizarlo en "funciones.py"

In [110]:
import pickle

In [111]:
with open("modelo_de_ML.pkl", "wb") as archivo:
    pickle.dump(modelo_entrenado, archivo)

In [119]:
with open ("modelo_de_ML.pkl", "rb") as archivo: 
    modelo_pickle = pickle.load(archivo)

In [115]:
modelo_pickle

### Hago la prediccion para mi modelo entrenado

In [58]:
predicciones = modelo_entrenado.test(testset)

In [59]:
predicciones

[Prediction(uid='76561198062110429', iid='DARK SOULS™: Prepare To Die™ Edition', r_ui=3.0, est=3.56906238270537, details={'was_impossible': False}),
 Prediction(uid='76561197970600478', iid='Orcs Must Die! Unchained', r_ui=4.0, est=4.24058300609127, details={'was_impossible': False}),
 Prediction(uid='76561198048992856', iid='Counter-Strike: Global Offensive', r_ui=5.0, est=4.586061871583415, details={'was_impossible': False}),
 Prediction(uid='foxer007', iid='Ori and the Blind Forest', r_ui=3.0, est=4.817853841742284, details={'was_impossible': False}),
 Prediction(uid='76561198031212089', iid="Sid Meier's Civilization® V", r_ui=5.0, est=4.557499846470229, details={'was_impossible': False}),
 Prediction(uid='OfficalUberZ', iid="Garry's Mod", r_ui=5.0, est=4.0348362479228586, details={'was_impossible': False}),
 Prediction(uid='76561198079378923', iid='Guns of Icarus Online', r_ui=5.0, est=4.352042088307719, details={'was_impossible': False}),
 Prediction(uid='DeZire26', iid='Spec Ops:

### Evaluamos las predicciones

In [60]:
accuracy.rmse(predicciones)

RMSE: 1.2863


1.2862619676160072

### Veo los juegos jugados por el usuario

In [71]:
juegos_jugados = merge[merge["user_id"] == "76561197970982479"]["app_name"].unique()

In [102]:
juegos_jugados

array(['Killing Floor', 'Zeno Clash'], dtype=object)

### Guardo todos los juegos en una variable

In [72]:
juegos = merge["app_name"].unique()

In [101]:
juegos

array(['Killing Floor', 'Zeno Clash', 'Euro Truck Simulator 2', ...,
       'The Journey Down: Chapter One', 'The Journey Down: Chapter Two',
       'Mystic Destinies: Serendipity of Aeons'], dtype=object)

### Crear una lista de juegos no valorados por el usuario específico

In [73]:
juegos_no_jugados = list(set(juegos) - set(juegos_jugados))

In [103]:
juegos_no_jugados

['Thea: The Awakening',
 'Demonicon',
 'Red Trigger',
 'Lucid9: Inciting Incident',
 'Fallout New Vegas®: Gun Runners’ Arsenal™',
 'Midnight Club 2',
 'Kindred Spirits on the Roof',
 'Flatout 3: Chaos & Destruction',
 'Dungeon of the Endless™',
 'Starseed Pilgrim',
 'Homefront',
 'Borderlands: The Pre-Sequel',
 'Half-Life 2 Soundtrack',
 'Epigenesis',
 'Kingdoms of Amalur: Reckoning™',
 'Imperial Glory',
 'Half Minute Hero: Super Mega Neo Climax Ultimate Boy',
 'Tokyo Hosto',
 '3079 -- Block Action RPG',
 'PAYDAY 2: The Official Soundtrack',
 'The Center - ARK Expansion Map',
 'GTGD S1: More Than a Gamer',
 'FINAL FANTASY TYPE-0™ HD',
 'The Graveyard',
 'RC Cars',
 'POSTAL 2',
 'STAR WARS™ Starfighter™',
 'Cosmic DJ',
 'Far Cry® 4',
 'Danganronpa 2: Goodbye Despair',
 'The Park',
 'Shadows on the Vatican Act I: Greed',
 'Hook',
 'Infinifactory',
 'Montas',
 'Heli Heroes',
 'F1 2015',
 'DeathSpank',
 'Mad Games Tycoon',
 'Jagged Alliance Flashback',
 'Dwarfs - F2P',
 'DG2: Defense Grid 

### Hacemos predicciones de los juegos no jugados por el usuario

In [116]:
predic_no_jugados = [modelo_pickle.predict("76561197970982479", linea) for linea in juegos_no_jugados]

In [104]:
predic_no_jugados

[Prediction(uid='76561197970982479', iid='Thea: The Awakening', r_ui=None, est=4.16589386178657, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Demonicon', r_ui=None, est=4.4175665436341465, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Red Trigger', r_ui=None, est=4.295972687899334, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Lucid9: Inciting Incident', r_ui=None, est=3.9955311444563195, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Fallout New Vegas®: Gun Runners’ Arsenal™', r_ui=None, est=4.250837482135692, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Midnight Club 2', r_ui=None, est=4.281275048264085, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Kindred Spirits on the Roof', r_ui=None, est=4.400422418706741, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Flato

### Ordeno la prediccion de acuerdo a su respectiva valoración y obtengo los 5 primeros

In [106]:
Top5_recomm = sorted(predic_no_jugados, key=lambda x: x.est, reverse=True)[:5] 

In [108]:
Top5_recomm

[Prediction(uid='76561197970982479', iid='Slime Rancher', r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='RimWorld', r_ui=None, est=4.996581905403162, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Transistor', r_ui=None, est=4.969867641577212, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Kingdom Rush', r_ui=None, est=4.924022418391751, details={'was_impossible': False}),
 Prediction(uid='76561197970982479', iid='Lethal League', r_ui=None, est=4.917278706140116, details={'was_impossible': False})]

### Metemos los juegos recomendados en una lista

In [95]:
lista_juegos = []
for recomendacion in Top5_recomm:
    lista_juegos.append(recomendacion.iid)

In [109]:
lista_juegos

['Slime Rancher', 'RimWorld', 'Transistor', 'Kingdom Rush', 'Lethal League']

### 5 juegos de recomendacion

In [98]:
{f"Juego 1:{lista_juegos[0]}, Juego 2:{lista_juegos[1]}, Juego 3:{lista_juegos[2]}, Juego 4:{lista_juegos[3]}, Juego 5:{lista_juegos[4]}"}

{'Juego 1:Slime Rancher, Juego 2:RimWorld, Juego 3:Transistor, Juego 4:Kingdom Rush, Juego 5:Lethal League'}

In [70]:
# ID del usuario para el cual quieres obtener recomendaciones
usuario_especifico = '76561197970982479'  # Reemplaza 'js41637' con el ID del usuario para el que quieres las recomendaciones

# Crear una lista de juegos ya valorados por el usuario específico
juegos_valorados = merge[merge['user_id'] == "76561197970982479"]['app_name'].unique()

# Crear una lista de todos los juegos disponibles
todos_los_juegos = merge['app_name'].unique()

# Crear una lista de juegos no valorados por el usuario específico
juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

# Generar predicciones para los juegos no valorados por el usuario
predicciones = [model.predict("MeloncraftLP", juego) for juego in juegos_no_valorados]

# Ordenar las predicciones en base a la valoración y obtener los juegos recomendados
recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5]  # Obtener las 5 mejores recomendaciones

# Mostrar los juegos recomendados
for recomendacion in recomendaciones:
    print(f"Juego: {recomendacion.iid}, Valoración estimada: {recomendacion.est}")

Juego: Rogue Legacy, Valoración estimada: 4.724344284486931
Juego: Bastion, Valoración estimada: 4.678700031439925
Juego: Lethal League, Valoración estimada: 4.665270323907626
Juego: DLC Quest, Valoración estimada: 4.65090407043316
Juego: Portal Stories: Mel, Valoración estimada: 4.621610898464593


In [ ]:
Juego: Slime Rancher, Valoración estimada: 5
Juego: RimWorld, Valoración estimada: 4.996581905403162
Juego: Transistor, Valoración estimada: 4.969867641577212
Juego: Kingdom Rush, Valoración estimada: 4.924022418391751
Juego: Lethal League, Valoración estimada: 4.917278706140116